# Introduction to PAML for dynamic systems
### Nonlinear spring pendulum

---

Lecture: "Physics-augmented machine learning" @ Cyber-Physical Simulation, TU Darmstadt

Lecturer: Prof. Oliver Weeger

Author: Fabian J. Roth

---

#### In this notebook, you will...

* <span style="color:red">learn something... Todo.</span>

In [ ]:
# Run this cell if you are using Google Colab
!git clone --depth 1 https://github.com/CPSHub/LecturePhysicsAwareML.git
%cd LecturePhysicsAwareML/PINNs
%pip install -e .

In [1]:
# Run this cell if you are working locally
%cd ..
%pip install -e .

c:\Users\roth\Documents\Git Repositories\LecturePhysicsAwareML_fork\dynamic_modeling
Note: you may need to restart the kernel to use updated packages.


c:\Users\roth\Documents\Git Repositories\LecturePhysicsAwareML_fork\dynamic_modeling\.venv\Scripts\python.exe: No module named pip


## 1. The spring pendulum system
In this task we consider a nonlinear spring pendulum system:

<img src="images\Spring_pendulum_only_cartesian.png" height="400"/>

The governing equations can be obtained by using Hamiltons Principle and the Euler---Lagrange equations.

Describing position of the mass $m$ with generalized (cartesian) coordinates $q_x, q_y$ and their velocities $\dot q_x, \dot q_y$, the total kinetic energy is given by
$$
\begin{align}
    T = \frac{1}{2}m \dot q_x^2 + \frac{1}{2}m \dot q_x^2.
\end{align}
$$

Similarly, the total potential energy is
$$
\begin{align}
    V = \frac{1}{2}c\big(l(\boldsymbol{q}) - l_0\big)^2 + mgq_y= \frac{1}{2}c\big(\sqrt{q_x^2+q_y^2} - l_0\big)^2 + mgq_y
\end{align}
$$

where $l$ and $l_0$ denote the spring length and spring rest length respectively, and $g$ is the gravitational constant.

We thus obtain the Lagrangian $\mathcal{L} = T - V$:
$$
\begin{align}
    \mathcal{L}(q_x, q_y, \dot q_x, \dot q_y) = \frac{1}{2}m(\dot q_x^2 + \dot q_y^2) - \left[ \frac{1}{2}c\left(\sqrt{q_x^2 + q_y^2} - l_0\right)^2 + mgq_y \right] + C
\end{align}
$$
where $C$ is an arbitrary constant corresponding to the chosen zero-level of the potential energy.

Applying the Euler-Lagrange equation results in the following equation of motion:
$$
\begin{align}
m\ddot q_x + c\left(1 - \frac{l_0}{\sqrt{q_x^2 + q_y^2}}\right)q_x &= 0\\
m\ddot q_y + c\left(1 - \frac{l_0}{\sqrt{q_x^2 + q_y^2}}\right)q_y + mg &= 0
\end{align}
$$
This can be written as a first order ordinary differential equation (ODE) by introducing the velocities $v_x=\dot q_x, v_y=\dot q_y$:

<span style="color:red">Todo.</span>

Let's generate some trajectories, using varying initial positions and zero initial velocity and with $l_0 = g = c =1$.

In [ ]:
from dynamic_modeling import ODESolver
import klax

import equinox as eqx
from jaxtyping import Array
import jax
import jax.numpy as jnp
from jax import random as jr
from jax.nn.initializers import variance_scaling

import matplotlib.pyplot as plt

class Derivative(eqx.Module):
    """
    Derivative function of a linear two mass oscillator system, with a
    force input u acting on the second mass.
    """

    A: Array
    B: Array
    J: Array
    R: Array
    Q: Array

    def __init__(self, m1, m2, k1, k2, d1, d2):
        zeros = jnp.zeros((2, 2))

        # Structure matrix
        mass = jnp.array([[m1, 0], [0, m2]])
        mass_inv = jnp.linalg.inv(mass)
        J = jnp.block([[zeros, mass_inv], [-mass_inv, zeros]])
        self.J = J

        # Resistive matrix
        diss = jnp.array(
            [
                [(d1 + d2) / (m1 * m1), -d2 / (m1 * m2)],
                [-d2 / (m1 * m2), d2 / (m1 * m2)],
            ]
        )
        R = jnp.block([[zeros, zeros], [zeros, diss]])
        self.R = R

        # Hamililtonian quadratic form H=xQx
        Q = jnp.array(
            [[k1 + k2, -k2, 0, 0], [-k2, k2, 0, 0], [0, 0, m1, 0], [0, 0, 0, m2]]
        )
        self.Q = Q

        self.A = (J - R) @ Q

        # Input matrix
        self.B = jnp.array([0, 0, 0, 1 / m2])[:, None]

    def __call__(self, t, y, u):
        return self.A @ y + self.B @ u